# Libraries

In [2]:
import numpy as np
import pandas as pd
import io
#import requests
import matplotlib.pyplot as plt
import warnings
from IPython.display import Markdown as md
import seaborn as sb
#import missingno as msno


warnings.filterwarnings('ignore')
# set to show all columns
pd.set_option('display.max_columns', 55)

ModuleNotFoundError: No module named 'seaborn'

# Declarations

In [ ]:
def plot_var(var='positive'):
    """
    Plots a bar chart of the given variable over the date range
    """
    assert type(var)==str, "Expected string as the variable name"

    y = df[var]
    x = df['date']
    plt.figure(figsize=(20,4))
    plt.title("Plot of \"{}\" for New York".format(var),fontsize=18)
    plt.plot(x,y,color='navy')
    plt.grid(False)
    plt.xticks(fontsize=14,rotation=45)
    plt.yticks(fontsize=14)
    plt.show()

def plot_hist(var='positiveIncrease'):
    """
    Plots a bar chart of the given variable over the date range
    """
    assert type(var)==str, "Expected string as the variable name"

    y = df[var]
    plt.figure(figsize=(20,4))
    plt.title("Plot of \"{}\" for New York".format(var),fontsize=18)
    plt.hist(y,color='royalblue')
    plt.grid(False)
    plt.xticks(fontsize=14,rotation=45)
    plt.yticks(fontsize=14)
    plt.show()

def add_sevenday_newCases(df):

    df['sevenDayAvg_newCases'] = 'NA'

    for i in range(0, len(df['positiveIncrease'])):
        if i < 7:
            df['sevenDayAvg_newCases'][i] = 0
        else :
            weekly = []
            for y in range(0,7):
                weekly.append(df['positiveIncrease'][i-y])
            df['sevenDayAvg_newCases'][i] = round(sum(weekly) / 7)
    return df

# 7 days moving average for hospitalization
def add_sevenday_hospitalize(df):

    df['sevenDayAvg_hospitalize'] = 'NA'

    for i in range(0, len(df['hospitalizedIncrease'])):
        if i < 7:
            df['sevenDayAvg_hospitalize'][i] = 0
        else :
            weekly = []
            for y in range(0,7):
                weekly.append(df['hospitalizedIncrease'][i-y])
            df['sevenDayAvg_hospitalize'][i] = round(sum(weekly) / 7)
    return df

# 7 days moving average for Death
def add_sevenday_death(df):

    df['sevenDayAvg_death'] = 'NA'

    for i in range(0, len(df['deathIncrease'])):
        if i < 7:
            df['sevenDayAvg_death'][i] = 0
        else :
            weekly = []
            for y in range(0,7):
                weekly.append(df['deathIncrease'][i-y])
            df['sevenDayAvg_death'][i] = round(sum(weekly) / 7)
    return df

# 7 days moving average for Test Result
def add_sevenday_testResult(df):

    df['sevenDayAvg_testResult'] = 'NA'

    for i in range(0, len(df['totalTestResultsIncrease'])):
        if i < 7:
            df['sevenDayAvg_testResult'][i] = 0
        else :
            weekly = []
            for y in range(0,7):
                weekly.append(df['totalTestResultsIncrease'][i-y])
            df['sevenDayAvg_testResult'][i] = round(sum(weekly) / 7)
    return df

# Import Data

In [ ]:
# API Hosting Suppose to End 5/1/2021
url="https://api.covidtracking.com/v1/states/ny/daily.cs" # API Call for NY data

try:
    s = requests.get(url).content
    df = pd.read_csv(io.StringIO(s.decode('utf-8')))

    if df.values.size < 3:  # ensure DF has values (may just contain 2 error values)
        raise FileNotFoundError("No data in API")
    else:
        df.to_csv('ny_covid_data.csv', index=False)  # future proof in case api goes down
        df = pd.read_csv(io.StringIO(s.decode('utf-8')))

except FileNotFoundError as fnf_error:
    df = pd.read_csv("ny_covid_data.csv")  # read from most recently fetched data
except: # catch any other unexpected error
    df = pd.read_csv("ny_covid_data.csv")

# Data Transformation

In [ ]:

# Format dates
df['date'] =  pd.to_datetime(df['date'], format='%Y%m%d') 

# Impute NaN values
# df.fillna(value=-1, inplace=True)

# apply filter on date
df = df[df['date'] <= '2020-06-30']
# df = df[df['date'] <= '2020-12-30']


# Datasource 3 (CDC)

In [ ]:
from requests import request
import json
from pandas.io.json import json_normalize
import pandas as pd
import datetime


response=request(url='https://data.cdc.gov/id/vbim-akqf.json?current_status=Laboratory-confirmed%20case', method='get')
covid_records_cdc = response.json()
cdc_df = pd.DataFrame(covid_records_cdc)
format_dt = '%Y-%m-%dT%H:%M:%S.%f'
format_dt2 = '%Y-%m-%d'
# Format dates
pd.to_datetime(cdc_df['cdc_case_earliest_dt'], format=format_dt)
pd.to_datetime(cdc_df['cdc_report_dt'], format=format_dt)
pd.to_datetime(cdc_df['pos_spec_dt'], format=format_dt)
cdc_df['cdc_case_earliest_dt']=pd.to_datetime(cdc_df['cdc_case_earliest_dt'], format=format_dt2)
cdc_df['cdc_report_dt']=pd.to_datetime(cdc_df['cdc_report_dt'], format=format_dt2)
cdc_df['pos_spec_dt']=pd.to_datetime(cdc_df['pos_spec_dt'], format=format_dt2)
cdc_df.head()


## Result: Since the data source doesn't provide the location of the patient, we won't be able to use it for our current analysis

# NYC Holiday calendar dataset for 2020


In [ ]:

holiday_data = {'date': ['2020-01-01','2020-01-20','2020-02-17','2020-05-25','2020-06-30','2020-09-07','2020-10-12','2020-11-03','2020-11-11','2020-11-26','2020-12-25'],
		'Holiday': ['New Year Day','Martin Luther King Jr. Day', 'Presidents Day', 'Memorial Day', 'Independence Day(Observed)','Labor Day','Columbus Day','Election Day','Veterans Day','Thanksgiving day','Christmas Day' ],
        'is_holiday':['1','1','1','1','1','1','1','1','1','1','1'],
		'is_long_weekend':['0','1','1','1','1','1','1','0','0','1','1']
		}

calendar_df = pd.DataFrame (holiday_data, columns = ['date','Holiday','is_holiday','is_long_weekend'])

calendar_df['date'] =  pd.to_datetime(calendar_df['date'], format='%Y-%m-%d')  
df2 = df.set_index('date').join(calendar_df.set_index('date'))

df2['is_holiday'].fillna('0',inplace=True)
df2['is_long_weekend'].fillna('0', inplace=True) 

df2.head()



## MTA travel stats for 2020

In [ ]:
mta_df = pd.read_csv('MTA_data.csv',index_col=[0], parse_dates=[0])


mta_df = mta_df[[ 'Subways: Total Estimated Ridership']] 
 
mta_df['Subways: Total Estimated Ridership'] = pd.to_numeric(mta_df['Subways: Total Estimated Ridership'])

mta_df.columns = ['Ridership']
mta_df.plot()

df2.join(mta_df)


# EDA

## Missing Data

There are columns that don't have a significant impact to our analysis, namely date checked and date modified features. These columns we want to exclude.

In [ ]:
df.drop(['dateChecked', 'dateModified'],axis=1,inplace=True)

We set a threshold of to tolerate up to 90% of values being NA per column. If a column contains more than 90% of NA values we drop it.

In [ ]:
df.info
limitPer = len(df) * .90
df = df.dropna(thresh=limitPer, axis=1)


There are also many features that just have zeros in them, we want to exclude that in our set as well.

In [ ]:
df = df.loc[:, (df != 0).any(axis=0)]

We ended up excluding 20 columns that just contained NA values, 2 columns that didn't have significant information (date modified and date checked), and 6 columns that only contained zeros.

## Feature Plots
In this first plot we can see the daily number of positive cases in NY from March 2020 - June 30, 2020:

In [ ]:
plot_var('positiveIncrease')

totalCases = "{:,.0f}".format(df['positiveIncrease'].sum())

md(f"In total there were {totalCases} positive cases in New York during that time range.")
# df.tail(n = 5)

The following plots also show the increases in hospitalizations, deaths, and the total amount of test results.

In [ ]:
plot_var('hospitalizedIncrease')

plot_var('deathIncrease')

plot_var('totalTestResultsIncrease')



In [ ]:
pd.set_option('display.max_columns', 56)

# 7 days moving average for positiveIncrease
df.sort_values(by=['date'], inplace=True, ascending=True)

# reset index
df.reset_index(level=0, inplace=True)

## Plots on 7-Day Moving Averages

In [ ]:
add_sevenday_newCases(df)
add_sevenday_hospitalize(df)
add_sevenday_death(df)
add_sevenday_testResult(df)

In [ ]:
plot_var('sevenDayAvg_newCases')
plot_var('sevenDayAvg_hospitalize')
plot_var('sevenDayAvg_death')
plot_var('sevenDayAvg_testResult')

## Histogram Plots

In [ ]:
plot_hist('positiveIncrease')

plot_hist('hospitalizedIncrease')

plot_hist('deathIncrease')

plot_hist('totalTestResultsIncrease')




## Correlation Matrix

In [ ]:
corrMatrix = df.corr()
fig, ax = plt.subplots(figsize=(10,10))
sb.heatmap(corrMatrix, annot=True, ax = ax)
plt.show()

It's plain to see that death increases are positively correlated with hospitalizations and positive cases. It would also make sense that there would exist a negative correlations between deaths and total tests done, as the more informed the population is the better course of action they can take based on their status results.

## Final State of data

In [ ]:
df.info

### Dataset 2 

Dataset provided by Johns Hopkins University Center for Systems Science and Engineering, here is github folder link for the dataset: https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series

There are 424 columns present in the dataset, and the dates are present as columns. We can apply pivot on dates and convert columns to rows. We will delete unnecessary columns such as iso2,iso3, Country_Region, Lat, Long_, FIPS, Admin2, Combined_Key. Apply filter on Province_State i.e New York and timeline from March 2020 to June 2020.

In [ ]:
# Get data
df_jhu = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv")
# View data
df_jhu.head()

In [ ]:
df_jhu_clean = pd.DataFrame(df_jhu)
# Filter by New york state
df_jhu_clean = df_jhu_clean[df_jhu_clean['Province_State'] == 'New York']
# Timeline from March to June 
df_jhu_clean = df_jhu_clean.loc[:, '3/1/20':'6/30/20']
# Check for Null value
df_jhu_clean.isnull().values.any()
# View Data
df_jhu_clean.head()

Apply reset index and calculate new cases from confirmed cases field.

In [ ]:
# List for Columns 
column_list = df_jhu_clean.columns.tolist()
# Convert columns to rows
df2 = pd.melt(df_jhu_clean, value_vars= column_list, value_name="Confirmed_Cases")
# Change column name to Date
df2.rename(columns = {'variable':'Dates'}, inplace = True)
# Sum confirm cased and group by date
df2 = pd.DataFrame(df2.groupby('Dates').sum())
# reset index
df2.reset_index(level=None, inplace=True) 
# Format Dates column
df2['Dates'] =  pd.to_datetime(df2['Dates'], format='%m/%d/%y')
# Order by Dates
df2.sort_values(by=['Dates'], inplace=True, ascending=True)
# View data
print(df2.head())

In [ ]:
# Reset Index
df2.reset_index(level=None, drop=True, inplace=True)

# Calculate New Cases per day
def New_Cases(df2):
    df2['New_Cases'] = 'NA'

    for i in range(0, len(df2["Confirmed_Cases"])):
        if i == 0:
            df2['New_Cases'][i] = 0
        elif i > 0:
            df2['New_Cases'][i]= df2['Confirmed_Cases'][i] - df2['Confirmed_Cases'][i-1] 
    return df2

    
# Call Function
New_Cases(df2)

In [ ]:
# Graph for Daily New cases
def line_graph():
    x = df2['Dates']
    y = df2['New_Cases']
    plt.figure(figsize=(20,4))
    plt.title("Plot of \"{}\" for New York".format("New Cases"),fontsize=18)
    plt.plot(x,y,color='navy')
    plt.grid(False)
    plt.xticks(fontsize=14,rotation=45)
    plt.yticks(fontsize=14)
    plt.show()
    
line_graph()

For daily Death and Recovered there are other files available in [this](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series) folder. If we use dataset 2 we need to go through data transformation. However, we are getting clean data from Tracking.com with all fields (daily cases, daily death, daily test result) in a single file, so we decide to use data from Tracking.com.